In [1]:
import pandas as pd
import altair as alt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


In [2]:
import mysql
from mysql.connector import connect, Error

cnx = mysql.connector.connect(user=f'{user}', password=f'{password}',
                              host=f'{host}', database=f"{database}")

cursor = cnx.cursor()

In [3]:
query = """
SELECT
count(SCA.TWEET_ID) AS TWEET_COUNT
,SC.SENTIMENT_CATEGORY
,CN.NAME_TEXT AS CATEGORY_NAME
FROM SENTIMENT_CATEGORY_ASSIGNMENT SCA
INNER JOIN SENTIMENT_CATEGORY SC ON SCA.SENT_ID = SC.SENT_ID
INNER JOIN TWEET_DATA TD ON TD.TWEET_ID = SCA.TWEET_ID
INNER JOIN SEARCHES S ON S.SEARCH_ID = TD.SEARCH_ID
INNER JOIN EVENTS E ON S.EVENT_ID = E.EVENT_ID
INNER JOIN CATEGORY C ON E.EVENT_ID = C.EVENT_ID
INNER JOIN CAT_NAME CN ON C.CAT_NAME_ID = CN.NAME_ID
GROUP BY
SC.SENTIMENT_CATEGORY
,CN.NAME_TEXT
"""

cat = pd.read_sql(query, cnx)

In [92]:
query2 = """
SELECT
count(SCA.TWEET_ID) as TWEET_COUNT
,SC.SENTIMENT_CATEGORY
,CN.NAME_TEXT AS CATEGORY_NAME
,E.EVENT_NAME
FROM SENTIMENT_CATEGORY_ASSIGNMENT SCA
INNER JOIN SENTIMENT_CATEGORY SC ON SCA.SENT_ID = SC.SENT_ID
INNER JOIN TWEET_DATA TD ON TD.TWEET_ID = SCA.TWEET_ID
INNER JOIN SEARCHES S ON S.SEARCH_ID = TD.SEARCH_ID
INNER JOIN EVENTS E ON S.EVENT_ID = E.EVENT_ID
INNER JOIN CATEGORY C ON E.EVENT_ID = C.EVENT_ID
INNER JOIN CAT_NAME CN ON C.CAT_NAME_ID = CN.NAME_ID
GROUP BY
SC.SENTIMENT_CATEGORY
,CN.NAME_TEXT
,E.EVENT_NAME
"""

catE = pd.read_sql(query2, cnx)

In [14]:
source = cat

alt.Chart(source).mark_bar().encode(
    x=alt.X('SENTIMENT_CATEGORY:N',title='',sort='-y',axis=alt.Axis(labels=False)),
    y=alt.Y('sum(TWEET_COUNT):Q', title='Count of Tweets'),
    color=alt.Color('SENTIMENT_CATEGORY:N', scale=alt.Scale(range=['#ed6300','#808080','#00d1e0']), legend= alt.Legend(title='Sentiment Category')),
    column=alt.Column('CATEGORY_NAME:O', title='Count of Sentiments by News Category')
).properties(
width = 100,
height = 300).configure_axis(
    grid=False    
)

alt.Chart(...)

In [98]:
catE[catE['EVENT_NAME']=='SAME SEX MARRIAGE']

,TWEET_COUNT,SENTIMENT_CATEGORY,CATEGORY_NAME,EVENT_NAME
1,475,NEGATIVE,CIVIL RIGHTS,SAME SEX MARRIAGE
6,475,NEGATIVE,HELATH,SAME SEX MARRIAGE
7,475,NEGATIVE,POLITICS,SAME SEX MARRIAGE
15,20,NEUTRAL,CIVIL RIGHTS,SAME SEX MARRIAGE
20,20,NEUTRAL,HELATH,SAME SEX MARRIAGE
21,20,NEUTRAL,POLITICS,SAME SEX MARRIAGE
29,1196,POSITIVE,CIVIL RIGHTS,SAME SEX MARRIAGE
34,1196,POSITIVE,HELATH,SAME SEX MARRIAGE
35,1196,POSITIVE,POLITICS,SAME SEX MARRIAGE


In [130]:
source1 = catE[catE['EVENT_NAME']!='SAME SEX MARRIAGE']
source2 = catE[catE['EVENT_NAME']=='SAME SEX MARRIAGE']

chart1 = alt.Chart(source1).mark_bar(color = '#808080').encode(
    x=alt.X('SENTIMENT_CATEGORY:N',title='',sort='-y'),
    y=alt.Y('sum(TWEET_COUNT):Q', title='Count of Tweets')
    # column='CATEGORY_NAME:O
)

chart2 = alt.Chart(source2).mark_bar(color = '#00ACC3').encode(
    x=alt.X('SENTIMENT_CATEGORY:N',title='',sort='-y'),
    y=alt.Y('sum(TWEET_COUNT):Q', title='Count of Tweets')
    # column='CATEGORY_NAME:O'
)

(chart1+chart2).properties(
width = 500,
height = 500    
)

alt.LayerChart(...)